In [ ]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from tqdm import tqdm

In [ ]:
from models import *


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=126, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print('==> Building model..')
net = ResNet152()

==> Building model..


In [ ]:
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

In [ ]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/'+"resnet152.net"+'-ckpt.t7')
        best_acc = acc


for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)


Epoch: 0
0 397 Loss: 2.461 | Acc: 8.730% (11/126)
1 397 Loss: 6.858 | Acc: 10.317% (26/252)
2 397 Loss: 9.381 | Acc: 10.847% (41/378)
3 397 Loss: 9.726 | Acc: 9.524% (48/504)
4 397 Loss: 10.048 | Acc: 9.683% (61/630)
5 397 Loss: 10.860 | Acc: 9.392% (71/756)
6 397 Loss: 10.873 | Acc: 9.751% (86/882)
7 397 Loss: 10.479 | Acc: 10.317% (104/1008)
8 397 Loss: 9.906 | Acc: 10.406% (118/1134)
9 397 Loss: 9.450 | Acc: 10.397% (131/1260)
10 397 Loss: 8.967 | Acc: 10.390% (144/1386)
11 397 Loss: 8.569 | Acc: 10.251% (155/1512)
12 397 Loss: 8.343 | Acc: 10.256% (168/1638)
13 397 Loss: 8.074 | Acc: 10.601% (187/1764)
14 397 Loss: 7.855 | Acc: 10.688% (202/1890)
15 397 Loss: 7.585 | Acc: 10.913% (220/2016)
16 397 Loss: 7.308 | Acc: 11.064% (237/2142)
17 397 Loss: 7.091 | Acc: 11.199% (254/2268)
18 397 Loss: 6.867 | Acc: 11.028% (264/2394)
19 397 Loss: 6.705 | Acc: 10.794% (272/2520)
20 397 Loss: 6.557 | Acc: 10.809% (286/2646)
21 397 Loss: 6.383 | Acc: 10.895% (302/2772)
22 397 Loss: 6.258 | Acc: